In [ ]:
!pip install langchain google-generativeai

In [ ]:
### 1.  calling Library:
import os
import json
import re
from langchain_core.runnables.base import RunnableSequence ### for serialization
from langchain.prompts import PromptTemplate
from typing import Optional, List
import google.generativeai as genai

In [ ]:
from typing import Optional, List
import google.generativeai as genai

class GeminiLLM:
    def __init__(self, model_name: str, api_key: str, generation_config: dict):
        """
        Initializes the GeminiLLM object with the provided model name, API key, and generation configuration.
        
        Args:
        - model_name: The name of the model to be used (e.g., 'gemini-2.0').
        - api_key: The API key used to authenticate with the Generative AI service.
        - generation_config: A dictionary containing configuration options for text generation.
        """
        self.model_name = model_name  # Store the model name
        self.api_key = api_key  # Store the API key
        self.generation_config = generation_config  # Store the generation configuration
        genai.configure(api_key=api_key)  # Configure the genai API with the provided API key

    def __call__(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        """
        Generates a response from the model based on the provided prompt.
        
        Args:
        - prompt: The text input that serves as the prompt for text generation.
        - stop: An optional list of stop sequences. If a stop sequence is found in the output, 
                the response will be truncated at that point.

        Returns:
        - A string containing the generated text.
        """
        # Call the text generation API with the provided parameters
        response = genai.generate_text(
            model=self.model_name,  # Specify the model to use
            prompt=prompt,  # Provide the prompt
            temperature=self.generation_config.get("temperature", 1),  # Control randomness in output
            top_p=self.generation_config.get("top_p", 0.95),  # Top-p sampling
            top_k=self.generation_config.get("top_k", 40),  # Top-k sampling
            max_output_tokens=self.generation_config.get("max_output_tokens", 8192),  # Limit output tokens
            response_mime_type=self.generation_config.get("response_mime_type", "text/plain"),  # Response format
        )
        
        # Check if the response has a 'text' attribute or 'generated_text', and assign it accordingly
        text = response.text if hasattr(response, 'text') else response.generated_text

        # If stop sequences are provided, truncate the output at the first stop sequence
        if stop:
            for stop_seq in stop:
                if stop_seq in text:
                    text = text.split(stop_seq)[0]  # Truncate at the first occurrence of a stop sequence

        return text  # Return the generated (and possibly truncated) text


In [ ]:
### 3. Load Recipes

def load_recipes(file_path):
    with open(file_path, 'r') as file:
        recipes = json.load(file)
    return recipes

In [ ]:
def search_recipes(query: str, recipes: list, top_k: int = 5):
    """
    Searches for recipes in a list based on a query. The function will return the top_k recipes 
    that match the query in their name, ingredients, or cuisine.

    Args:
    - query: The search query string (e.g., an ingredient, dish name, or cuisine).
    - recipes: A list of recipes where each recipe is represented as a dictionary with keys like 'name', 'ingredients', and 'cuisine'.
    - top_k: The maximum number of matching recipes to return (default is 5).

    Returns:
    - A list of recipes that match the search query.
    """
    query = query.lower()  # Convert the query to lowercase to make the search case-insensitive
    results = []  # Initialize an empty list to store matching recipes

    # Iterate over each recipe in the list of recipes
    for recipe in recipes:
        name = recipe['name'].lower()  # Convert the recipe name to lowercase
        ingredients = ' '.join(recipe['ingredients']).lower()  # Join ingredients and convert to lowercase
        cuisine = recipe['cuisine'].lower()  # Convert the cuisine to lowercase

        # Check if the query matches any part of the recipe (name, ingredients, or cuisine)
        if (query in name) or (query in ingredients) or (query in cuisine):
            results.append(recipe)  # If match is found, add the recipe to results

        # Stop early if we have already found top_k matches
        if len(results) >= top_k:
            break

    return results  # Return the list of matching recipes


In [ ]:
def setup_pipeline(model_name: str, api_key: str, generation_config: dict):
    """
    Sets up a Retrieval-Augmented Generation (RAG) pipeline with a given model and configuration.
    This pipeline allows for retrieving relevant recipes based on a user query and generating 
    a detailed response using a language model.

    Args:
    - model_name: The name of the AI model (e.g., 'gemini-2.0').
    - api_key: The API key to authenticate the request to the Generative AI service.
    - generation_config: A dictionary of configuration options for the language model (e.g., temperature, top_k, etc.).

    Returns:
    - A callable pipeline function that takes a query and a list of recipes, then generates a detailed response.
    """
    
    # Initialize the GeminiLLM class with the model name, API key, and generation configuration
    gemini_llm = GeminiLLM(
        model_name=model_name,  # Specify the model to use (e.g., gemini-2.0)
        api_key=api_key,  # API key for authentication
        generation_config=generation_config  # Configuration options for text generation
    )

    # Define the prompt template to structure the user query and retrieved recipes for the LLM
    prompt_template = PromptTemplate(
        input_variables=["recipes", "query"],  # Define the placeholders to be used in the template
        template="""
            You are a culinary expert. Based on the following recipes, answer the user's query in detail.

            ### Retrieved Recipes:
            {recipes}

            ### User Query:
            {query}

            ### Response:
        """
    )

    # Wrap the gemini_llm in a lambda function so it can be invoked in the pipeline sequence
    # The lambda takes the input dictionary (which will contain the 'query') and sends it to the gemini_llm for generation
    pipeline = RunnableSequence([prompt_template, lambda x: gemini_llm(prompt=x['query'])])

    # Return the constructed pipeline function that combines the prompt template and model response generation
    return pipeline


In [ ]:
def get_recipe_suggestion(user_query: str, recipes: list, pipeline: RunnableSequence):
    """
    This function takes a user query, searches for matching recipes, and generates a recipe suggestion
    using a predefined RAG pipeline. If no matching recipes are found, it provides a general suggestion.

    Args:
    - user_query: The user's query or request for recipe suggestions.
    - recipes: A list of available recipes (each recipe is a dictionary containing 'name', 'ingredients', and 'cuisine').
    - pipeline: The RAG pipeline that generates a response using the recipes and query.

    Returns:
    - A response generated by the pipeline based on the query and matching recipes.
    """
    
    # Search for matching recipes based on the user query (top 5 results)
    matching_recipes = search_recipes(user_query, recipes, top_k=5)

    if not matching_recipes:
        # If no recipes are found, create a fallback message indicating no specific recipes were found
        recipes_text = "No specific recipes were found, but please provide a general suggestion based on the query."
    else:
        # If matching recipes are found, format them into a text string
        recipes_text = "\n\n".join(
            # For each matching recipe, format the name, ingredients, and cuisine
            f"**Name**: {recipe['name']}\n**Ingredients**: {', '.join(recipe['ingredients'])}\n**Cuisine**: {recipe['cuisine']}"
            for recipe in matching_recipes
        )

    # Run the pipeline to generate a response, passing the recipes (or fallback message) and the user query
    response = pipeline.invoke({"recipes": recipes_text, "query": user_query})
    
    # Return the generated response from the pipeline
    return response


In [ ]:
def main():
    """
    Main function that sets up the pipeline, loads the recipe data, and processes a user query
    to generate a recipe suggestion using the Gemini language model.
    """
    
    # Define the model name that will be used for text generation
    model_name = "gemini-2.0-flash-exp"
    
    # API key for authenticating the request to the AI model (replace with actual API key)
    api_key = ""  # Replace with your actual API key
    
    # Define the configuration settings for the language model's response generation
    generation_config = {
        "temperature": 0.7,  # Controls randomness in the output (0.7 is balanced)
        "top_p": 0.9,        # Controls the diversity of the output (higher value means less diversity)
        "top_k": 50,         # Controls the number of possible next tokens to sample from (higher value increases options)
        "max_output_tokens": 1500,  # Sets the maximum number of tokens in the output (response length)
        "response_mime_type": "text/plain",  # Specifies the format of the response (plain text)
    }

    # Load the recipe data from a file ('recipes.json' is assumed to be the file containing the recipes)
    recipes = load_recipes("recipes.json")
    
    # Set up the pipeline by passing the model name, API key, and generation configuration
    pipeline = setup_pipeline(model_name, api_key, generation_config)

    # Define the user query for the recipe suggestion
    user_query = "Suggest a recipe using chicken and garlic that fits Italian cuisine."

    # Get the recipe suggestion by passing the user query, recipe data, and the pipeline
    response = get_recipe_suggestion(user_query, recipes, pipeline)
    
    # Print the response from the Gemini model, which is the recipe suggestion
    print("Gemini's Recipe Suggestion:\n")
    print(response)

# Ensure the main function is executed when the script is run directly
if __name__ == "__main__":
    main()
